In [1]:
!pip install PyDrive

     |████████████████████████████████| 993kB 2.8MB/s 
  Created wheel for PyDrive: filename=PyDrive-1.3.1-cp36-none-any.whl size=27435 sha256=3d5f93951788d8d9db267fc0efeccf1e04e945cf8e72b36ef9cd77ec7c9c18c3
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({"id": "1Dn8h7wDHf4WWS4_vKNHyJ7qUVer3Aumx"}) #이부분 수정하면 됩니당
download.GetContentFile("ratings_train.txt")

In [5]:
############Konlpy 설치 ##########################

!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install JPype1-py3
!pip3 install konlpy
!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Packages [13.2 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13

In [0]:
def read_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        # txt 파일의 헤더(id document label)는 제외하기
        data = data[1:]
    return data

train_data = read_data('ratings_train.txt')

In [0]:
train_corpus = [z[1]  for z in train_data]
y_train = [z[2]  for z in train_data]
del train_data

In [0]:
from konlpy.tag import Twitter
from tqdm import trange

In [9]:
twitter = Twitter()

/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
/usr/local/lib/python3.6/dist-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [10]:
generator = (i for i in train_corpus)
sentences = []
for _ in trange(len(train_corpus)):
  temp = next(generator)
  tokens = twitter.morphs(temp)
  sentences.append(tokens)

100%|██████████| 150000/150000 [05:58<00:00, 418.13it/s]


In [0]:
import numpy as np
import re
import itertools
from collections import Counter
import pandas as pd

In [0]:
def pad_sentences(sentences, padding_word="<PAD/>", sequence_length= 115):
    """
    Pads all sentences to the same length. The length is defined by the longest sentence.
    Returns padded sentences.
    """
    padded_sentences = []
    for i in range(len(sentences)):
        sentence = sentences[i]
        num_padding = sequence_length - len(sentence)
        new_sentence = sentence + [padding_word] * num_padding
        
        if len(new_sentence) > 115:
            new_sentence = new_sentence[:115]
        padded_sentences.append(new_sentence)
    
    return padded_sentences
  
def build_vocab(sentences):
    """
    Builds a vocabulary mapping from word to index based on the sentences.
    Returns vocabulary mapping and inverse vocabulary mapping.
    """
    # Build vocabulary
    word_counts = Counter(itertools.chain(*sentences))
    # Mapping from index to word
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    vocabulary_inv = list(sorted(vocabulary_inv))
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return [vocabulary, vocabulary_inv]
  
def build_input_data(sentences, labels, vocabulary):
    """
    Maps sentences and labels to vectors based on a vocabulary.
    """
    x = np.array([[vocabulary[word] for word in sentence] for sentence in sentences])
    y = np.array(labels)
    return [x, y]

In [0]:
labels = pd.get_dummies(y_train).values

In [0]:
sentences_padded = pad_sentences(sentences)
vocabulary, vocabulary_inv = build_vocab(sentences_padded)
x, y = build_input_data(sentences_padded, labels, vocabulary)

In [15]:
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.4
                                                    , random_state=201, stratify = y)

In [32]:
X_train

array([[ 82810, 102162,   6499, ...,   2722,   2722,   2722],
       [ 64160,  33621,  74541, ...,   2722,   2722,   2722],
       [ 21451,  21451,  83981, ...,   2722,   2722,   2722],
       ...,
       [103309,  48264,  67740, ...,   2722,   2722,   2722],
       [ 25920,  67775,  18393, ...,   2722,   2722,   2722],
       [ 57861,    703,   2722, ...,   2722,   2722,   2722]])

In [0]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, SimpleRNN
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)

In [0]:
sequence_length = x.shape[1] # 115
vocabulary_size = len(vocabulary_inv) # 104679
embedding_dim = 256
epochs = 3
batch_size = 30

In [41]:
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_dim, input_length=sequence_length))
model.add(SimpleRNN(100))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 115, 256)          26797824  
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 100)               35700     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total params: 26,833,726
Trainable params: 26,833,726
Non-trainable params: 0
_________________________________________________________________
None
Train on 90000 samples, validate on 60000 samples
Epoch 1/3
90000/90000 [==============================] - 206s 2ms/step - loss: 0.6987 - acc: 0.5011 - val_loss: 0.6982 - val_acc: 0.4881
Epoch 2/3
90000/90000 [==============================] - 201s 2ms/step - loss: 0.6964 - acc: 0.5034 - val_loss: 0.6943 - val_acc: 0.4873
Epoch 3/3
90000/90000 [==============================] - 200s 2ms